<a href="https://colab.research.google.com/github/DonRoboto/MachineLearning/blob/master/NaiveBayes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detector de Spam

In [23]:
# Importar la libreria Pandas 
import pandas as pd

# Dataset de https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_table('sample_data/SMSSpamCollection', 
                   sep='\t', 
                   names=['label','sms_message'])

# Visualización de las 5 primeras filas
df.head(10)

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


convertiremos las etiquetas en variables binarias, 0 representará ‘ham’ y 1 representará ‘spam’.

In [24]:
# Conversion
#df['label'] = df.label.map({'ham':0, 'spam':1})

# Visualizar las dimensiones de los datos
#df

# crear un dict file 
clase = {'ham': 0,'spam': 1}

df.label = [clase[item] for item in df.label]

df


,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


“Bag of Words” (BoW)

In [25]:
# Definir los documentos
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

# Importar el contador de vectorizacion e inicializarlo
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

# Visualizar del objeto'count_vector' que es una instancia de 'CountVectorizer()'
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


Nombres de características para este conjunto de datos, que es el conjunto de palabras que componen nuestro vocabulario para “documentos”.

In [26]:
count_vector.fit(documents)
names = count_vector.get_feature_names()
names

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

crear una matriz cuyas filas serán una de cada cuatro documentos, y las columnas serán cada palabra. El valor correspondiente (fila, columna) será la orecuencia de ocurrencia de esa palabra (en la columna) en un documento particular (en la fila).

In [27]:
doc_array = count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

Para hacerlo fácil de entender, nuestro paso siguiente es convertir esta tabla en una estructura de datos y nombrar las columnas adecuadamente.

In [28]:
frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


Dividiendo el Conjunto de Datos en Conjuntos de Entrenamiento y Pruebas

In [29]:
# Dividir los datos en conjunto de entrenamiento y de test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], df['label'], random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


Convertir nuestros datos al formato de la matriz buscada

In [30]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

Implementación Naive Bayes con Sci-Kit Learn

Usaremos la implementación Naive Bayes “multinomial”.

In [31]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()

naive_bayes.fit(training_data, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

nuestro algoritmo ha sido entrenado usando el conjunto de datos de entrenamiento, podemos hacer algunas predicciones en los datos de prueba almacenados en ‘testing_data’ usando predict()

In [32]:
predictions = naive_bayes.predict(testing_data)

Evaluación del modelo

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


Nuevos casos
https://www.thatsnonsense.com/6-spam-text-message-examples-in-2021/

In [64]:

X_new=["Apple Store: There has been a purchase made on your account.",
       "Hi!. I'm waiting for your call",
       "(Coinbase) Amount recieved 2.221 Bitcoin BTC ($18,421 USD) Please confirm transaction: http://bit.do/Coinbase432194-53242",
       "NHS: We have identified that your are elegible to apply for vaccine. For more information and to apply, follow here: application-ukform.com" ]

new_obs = count_vector.transform(X_new)

new_prediction = naive_bayes.predict(new_obs)

new_prediction

array([1, 0, 1, 1])